# Coursera Capstone Project

## Purpose:
This notebook is going to be used to complete the Coursera Capstone Project for the IBM Data Science Professional Certificate.

In [ ]:
impo

In [ ]:
import numpy as np